## Trying to understand and improve AuxTel ISR

Craig Lage - 04-Jan-23

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
%matplotlib inline

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask

In [ ]:
from lsst.daf.butler import Butler
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib"])

In [ ]:
def plotExp(exp):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure
    
def repairCosmics(postIsr):
    # Borrowed from summit utils
    if postIsr.getPsf() is None:
        installPsfTask = InstallGaussianPsfTask()
        installPsfTask.run(postIsr)

    # TODO: try adding a reasonably wide Gaussian as a temp PSF
    # and then just running repairTask on its own without any
    # imChar. It should work, and be faster.
    repairConfig = CharacterizeImageTask.ConfigClass()
    repairConfig.doMeasurePsf = False
    repairConfig.doApCorr = False
    repairConfig.doDeblend = False
    repairConfig.doWrite = False
    repairConfig.repair.cosmicray.nCrPixelMax = 200000
    repairTask = CharacterizeImageTask(config=repairConfig)
    repairTask.repair.run(postIsr)
    return

# First, try to reproduce what we see in RubinTV

With the settings below, it matches very well.

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.leadingColumnsToSkip = 5
isrConfig.overscan.doParallelOverscan=False
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=True
isrConfig.doStrayLight=False
isrConfig.doFlat=True
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=True
isrConfig.doNanMasking=True
isrConfig.doInterpolate=True
isrConfig.doSaturation=True
isrConfig.doSaturationInterpolation=True
isrConfig.doWrite=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
expId=2022121200752
exp = butler.get('raw', detector=0, exposure=expId)
biasExp = butler.get('bias', detector=0, exposure=expId)
flatExp = butler.get('flat', detector=0, exposure=expId)
darkExp = butler.get('dark', detector=0, exposure=expId)
defectExp = butler.get('defects', detector=0, exposure=expId)
isrResult = isrTask.run(exp, bias=biasExp, flat=flatExp, dark=darkExp, defects=defectExp)
postIsrExp = isrResult.exposure
repairCosmics(postIsrExp)
fig = plotExp(postIsrExp)
plt.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/Baseline_{expId}.png")

In [ ]:
pixelHist = postIsrExp.image.array.flatten()
pixelHist = np.clip(pixelHist, -40.0,40.0)
mean = np.mean(pixelHist)
std = np.std(pixelHist)
plt.title("Baseline ISR pixel distribution")
plt.hist(pixelHist, range=(-40,40))
plt.text(-30.0, 5.0E6, f"Mean = {mean:.2f}")
plt.text(-30.0, 4.0E6, f"Std = {std:.2f}")
plt.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/Baseline_Sigma_06Jan23.pdf")

# Why doesn't the bias subtraction remove the dark lines?

In [ ]:
plotArray = np.median(postIsrExp.image.array[2500:3500, 0:1500], axis=0)
plt.subplot(2,1,1)
plt.plot(plotArray)
plt.subplot(2,1,2)
plt.plot(plotArray[450:530])

# Next, implement ISR one at a time

In [ ]:
isrConfig.doBias=False
isrConfig.doDark=False
isrConfig.doFlat=False
isrConfig.doDefect=False
isrTask = IsrTask(config=isrConfig)
isrResult = isrTask.run(exp, bias=biasExp, flat=flatExp, dark=darkExp, defects=defectExp)
postIsrExp = isrResult.exposure
repairCosmics(postIsrExp)
fig = plotExp(postIsrExp)
plt.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/Overscan_Only_{expId}.png")

In [ ]:
isrConfig.doBias=True
isrConfig.doDark=False
isrConfig.doFlat=False
isrConfig.doDefect=False
isrTask = IsrTask(config=isrConfig)
isrResult = isrTask.run(exp, bias=biasExp, flat=flatExp, dark=darkExp, defects=defectExp)
postIsrExp = isrResult.exposure
repairCosmics(postIsrExp)
fig = plotExp(postIsrExp)
plt.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/Overscan_Bias_{expId}.png")

In [ ]:
isrConfig.doBias=True
isrConfig.doDark=True
isrConfig.doFlat=False
isrConfig.doDefect=False
isrTask = IsrTask(config=isrConfig)
isrResult = isrTask.run(exp, bias=biasExp, flat=flatExp, dark=darkExp, defects=defectExp)
postIsrExp = isrResult.exposure
repairCosmics(postIsrExp)
fig = plotExp(postIsrExp)
plt.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/Overscan_Bias_Dark_{expId}.png")

In [ ]:
isrConfig.doBias=True
isrConfig.doDark=True
isrConfig.doFlat=False
isrConfig.doDefect=True
isrTask = IsrTask(config=isrConfig)
isrResult = isrTask.run(exp, bias=biasExp, flat=flatExp, dark=darkExp, defects=defectExp)
postIsrExp = isrResult.exposure
repairCosmics(postIsrExp)
fig = plotExp(postIsrExp)
plt.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/Overscan_Bias_Dark_Defect_{expId}.png")

In [ ]:
isrConfig.doBias=True
isrConfig.doDark=True
isrConfig.doFlat=True
isrConfig.doDefect=True
isrTask = IsrTask(config=isrConfig)
isrResult = isrTask.run(exp, bias=biasExp, flat=flatExp, dark=darkExp, defects=defectExp)
postIsrExp = isrResult.exposure
repairCosmics(postIsrExp)
fig = plotExp(postIsrExp)
plt.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/Overscan_Bias_Dark_Defect_Flat_{expId}.png")